# Agentic RAG: Implementação utilizando CrewAI

Agentic RAG é a implementação de um processo de RAG baseado em **agentes**. A capacidade de raciocínio e tomada de decisão dos agentes, geralmente construídos na arquitetura ReAct, pode melhorar significativamente o processo em relação ao RAG padrão. Algumas vantagens dessa estratégia são:

* **Flexibilidade nas fontes de dados:** Os agentes têm a capacidade de entender profundamente a necessidade e trabalhar com múltiplas fontes de dados, sejam dados textuais estáticos, bancos relacionais ou até mesmo a internet.
* **Resolução de erros:** Os agentes podem lidar com problemas de forma efetiva, automatizando retries, mudando estratégias e pedindo esclarecimentos ao usuário.
* **Contexto:** Em um RAG agêntico, há a consideração de interações anteriores e a manutenção de uma conversação humanizada e satisfatória.

Este notebook demonstra o processo de criação de um RAG agêntico utilizando o CrewAI como orquestrador, além de algumas ferramentas do LangChain.

O resultado final é um chat que pode ser utilizado como apoio para funcionários da área de suporte técnico de uma empresa. Nele, é possível consultar informações sobre tickets no sistema (por meio de um banco de dados SQL) ou orientações sobre como lidar com certos problemas ou situações (por meio de um manual PDF).

## 1 - Importando as bibliotecas necessárias

As bibliotecas utilizadas neste projeto são:

* **CrewAI**: Framework para a construção de sistemas multi-agênticos. Permite criar agentes, designar tarefas e montar agentes em uma equipe para realizar as tarefas por meio das ferramentas disponibilizadas.

* **LangChain**: Framework com diversas aplicações no contexto das LLMs. Neste projeto, foi utilizado para disponibilizar ferramentas à equipe do CrewAI. O SQL Database Toolkit permite a interação com bancos de dados, e o FAISS é uma base vetorial na memória que permite RAG semântico em texto. Além disso, a integração do LangChain com o AWS Bedrock foi utilizada para definir a LLM (Claude 3.5 Sonnet) e o modelo de vetorização.

* **Pandas**: Utilizado para ler o arquivo CSV que contém os dados dos tickets.

* **sqlite3**: Utilizado para criar um banco de dados temporário com base no arquivo CSV.

In [ ]:
from crewai import Agent, Task, Crew
from crewai.tasks.conditional_task import ConditionalTask
import sqlite3
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.agent_toolkits.sql.toolkit import SQLDatabaseToolkit
import pandas as pd
from langchain_aws import ChatBedrock
from crewai_tools import tool
from langchain_community.embeddings import BedrockEmbeddings
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from crewai.tasks.task_output import TaskOutput
from pydantic import BaseModel
from IPython.display import display

## 2 - Criando o banco de dados

Para simular um banco de dados contendo informações sobre tickets em um sistema, foi utilizado o seguinte dataset, disponível publicamente no Kaggle:

[Customer Support Ticket Dataset](https://www.kaggle.com/datasets/suraj520/customer-support-ticket-dataset)

Esta célula realiza as seguintes operações:

1. Lê o arquivo CSV do dataset.
2. Cria um banco de dados temporário do SQLite3.
3. Insere os dados do arquivo CSV como uma tabela no banco de dados.

In [ ]:
csv_file_path = 'customer_support_tickets.csv'
db_file_path = 'tickets.db'
db_uri = f"sqlite:///{db_file_path}"

df = pd.read_csv(csv_file_path)
display(df.head(5))

with sqlite3.connect(db_file_path) as conn:
    df.to_sql('Tickets', conn, if_exists='replace', index=False)

db = SQLDatabase.from_uri(db_uri)
print("Database connection and table creation successful.")


,Ticket ID,Customer Name,Customer Email,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Resolution,Ticket Priority,Ticket Channel,First Response Time,Time to Resolution,Customer Satisfaction Rating
0,1,Marisa Obrien,carrollallison@example.com,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Social media,2023-06-01 12:15:36,NaN,NaN
1,2,Jessica Rios,clarkeashley@example.com,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,NaN,Critical,Chat,2023-06-01 16:45:38,NaN,NaN
2,3,Christopher Robbins,gonzalestracy@example.com,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Case maybe show recently my computer follow.,Low,Social media,2023-06-01 11:14:38,2023-06-01 18:05:38,3.0
3,4,Christina Dillon,bradleyolson@example.org,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Try capital clearly never color toward story.,Low,Social media,2023-06-01 07:29:40,2023-06-01 01:57:40,3.0
4,5,Alexander Carroll,bradleymark@example.com,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,West decision evidence bit.,Low,Email,2023-06-01 00:12:42,2023-06-01 19:53:42,1.0


Database connection and table creation successful.


## 3 - Disponibilizando a LLM e criando ferramenta de queries SQL

Neste projeto, foi utilizado o AWS Bedrock como provedor da LLM. O modelo escolhido foi o Anthropic Claude 3.5 Sonnet, devido à sua alta capacidade de lidar com problemas complexos. O método `ChatBedrock` do `langchain-aws` define a LLM.

No CrewAI, é possível disponibilizar ferramentas para os agentes. Essas ferramentas podem ser nativas ou importadas do LangChain ou LlamaIndex. Foi utilizado o SQL Database Toolkit do LangChain para que os agentes pudessem interagir com o banco de dados.

In [ ]:
llm = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0")
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
toolkit.get_tools()

[QuerySQLDataBaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019893725940>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019893725940>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000019893725940>),
 QuerySQLCheckerTool(description='Use this tool to 

## 4 - Criando base vetorial

Uma base vetorial permite armazenar documentos vetorizados e realizar uma busca nesses documentos por meio de similaridade de vetores. Esta célula insere o documento "manual.pdf" na base com os seguintes passos:

1. Inicializa o modelo de vetorização.
2. Carrega o PDF por meio do `PyPDFLoader` do LangChain.
3. Divide o PDF em chunks por meio do text splitter do LangChain. Essa divisão separa o arquivo em partes menores, otimizando o futuro processo do RAG.
4. Vetoriza os chunks do documento e os armazena em uma base vetorial FAISS.
5. Testa a capacidade de retrieval da base vetorial.

In [ ]:
# 1
embeddings = BedrockEmbeddings()

# 2
loader = PyPDFLoader("manual.pdf")
docs = loader.load()

# 3
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# 4
db = FAISS.from_documents(splits, embeddings)
retriever = db.as_retriever()

# 5
relevant_docs = retriever.invoke(input="What to do when the user reports missing credits?")
relevant_docs

[Document(metadata={'source': 'manual.pdf', 'page': 12}, page_content='Escalation:  \n- Persistent training issues should be escalated to the Machine Learning \nEngineering team  \n \n3.9 Billing and Invoice Discrepancies  \nDescription: Users report incorrect charges, missing credits, or unclear invoice \nitems.  \n \nTroubleshooting steps:'),
 Document(metadata={'source': 'manual.pdf', 'page': 11}, page_content="Troubleshooting steps:  \na) Identify the user's browser, version, and operating system  \nb) Reproduce the issue in multiple environments  \nc) Check for JavaScript console errors  \nd) Review recent frontend deployments  \ne) Verify CSS and asset loading  \n \nResolution:  \n- Provide browser -specific troubleshooting steps (cache clearing, etc.)  \n- Offer alternative browsers or versions known to work  \n- Apply hotfixes for confirmed bugs  \n \nEscalation:  \n- Report reproducible UI bugs to the Frontend Development team  \n \n3.7 Account Management Issues  \nDescription

## 5 - Criando ferramenta de busca vetorial

No CrewAI, também é possível criar ferramentas personalizadas. Embora já exista uma ferramenta nativa do CrewAI para busca vetorizada em PDFs, até o momento do desenvolvimento deste projeto, não havia integração com os modelos de vetorização da AWS. Portanto, foi criada uma ferramenta personalizada que utiliza a base vetorial previamente criada para retornar passagens úteis dos documentos armazenados na base, de acordo com a pergunta de input.

In [ ]:
@tool("SearchPDFTool")
def SearchPDFTool(question: str) -> str:
    """This tool takes a question and returns relevant chunks from the provided PDF that may be useful to answer the question."""
    documents = retriever.invoke(input=question)
    return str(documents)

## 6 - Criando os agentes

Um agente do CrewAI pode ter os seguintes atributos:

* **Role**: O papel do agente.
* **Goal**: O objetivo a ser cumprido.
* **Backstory**: Descrição mais detalhada das responsabilidades e capacidades do agente.
* **Tools**: Uma lista de ferramentas que podem ser utilizadas.

Neste projeto, foram concebidos 2 agentes:

1. **Senior Data Retriever**: Trabalha diretamente com as fontes de dados.
2. **Internal Support Specialist**: Interage diretamente com o usuário.

Ambos os agentes colaboram para atingir os objetivos do processo.

In [ ]:
retriever_agent = Agent(
    role='Senior Data Retriever',
    goal='Based on a question, retrieve and select the raw data that can be useful for answering the question.',
    backstory=
    'An experienced data retriever that has deep uderstanding of its companys database and text files, able to find and retrieve the data that can be used to answer any question in the companys context. Always adopts the strategy of querying first the data source thats more likely to have useful information. If that query result is already enough to answer the question, does not query any other data sources, as to get his job done faster. If not, queries the nex data source, and so on.',
    tools=[SearchPDFTool, *toolkit.get_tools()],
    verbose=True,
    llm=llm)

writer_agent = Agent(
    role='Internal Support Specialist',
    goal='Handle the conversation with the employee, interpreting the employees request and answering the request with the available data. Dont answer questions that dont fit into the internal support context.',
    backstory=
    'An experienced support specialist that knows exactly how to interpret the conversation with the employee, use the data available to answer questions and deal with conversations. Likes to be sympathetic but avoids overwriting and goes straight to the point.',
    verbose=True,
    llm=llm)

## 7 - Criando as tarefas

A próxima etapa é criar tarefas para serem cumpridas pelos agentes. Uma tarefa é definida por uma descrição detalhada, o agente encarregado, o output esperado, entre outros atributos.

Para o RAG agêntico, foram concebidas 3 tarefas:

1. **summarize_task**:
   - Objetivo: Entender o input do usuário.
   - Ação: Se for uma pergunta válida dentro do contexto, enviar ao data retriever. Caso contrário, pular para a última tarefa (que responde ao usuário).
   - Responsável: Internal Support Specialist.

2. **get_data_task**:
   - Objetivo: Retornar os dados brutos que podem ser utilizados para responder à pergunta.
   - Observação: Esta tarefa é **opcional**. Só será realizada caso o Internal Support Specialist, na tarefa summarize_task, entenda que o input do usuário é uma pergunta válida. Isso evita que o data retriever receba a tarefa de responder a inputs como "Boa tarde", por exemplo.
   - Responsável: Senior Data Retriever.

3. **answer_task**:
   - Objetivo: Gerar uma resposta ao usuário.
   - Ação: Basear-se na pergunta e nos dados de referência (se existentes) para formular a resposta.
   - Responsável: Internal Support Specialist.

In [ ]:
def is_question(output: TaskOutput) -> bool:
    return len(output.pydantic.answer) > 1

class EventOutput(BaseModel):
    answer: str

summarize_task = Task(
    description= """Interpret the user's input: "{question}". Decide whether to pass the question to the data retriever to get references to answer the question or not. If the user's input is a valid question, output the question. If it isn't, and therefore the data retriever shouldn't be contacted, simply output an empty string.
    """,
    agent=writer_agent,
    output_pydantic=EventOutput,
    expected_output="The question, or an empty string if retrieving information is not the case.")

get_data_task = ConditionalTask(
    description= """Based on the question "{question}", decide where to find the data that can be used to answer the question and then retrieve it. The data sources you have access to are:
    Support Tickets Database: A database containing information regarding recent or past customer support tickets, accessible through the SQL tools. Used only to query about tickets. Shouldn't be used to get information on how to deal with problems or situations.
    Customer Support Manual: A PDF manual containing information on how to deal with common problems, common scenarios etc. Available through the PDF tool. Used for questions on how to deal with situations, what to do etc.
    Best practice: Query the data source that is more likely to have information to answer the question first. If the results are already enough, do not query another data source, as to decrease the employee's waiting time.
    """,
    agent=retriever_agent,
    condition=is_question,
    expected_output="The raw data that can be used to formulate an answer to the question.")

answer_task = Task(
    description= """Based on the input "{question}", formulate a satisfying answer to the employee. If it is a question and data was retrieved in the get_data_task, the answer should be 100 percent based on the available references, without making up any infomation. If you can't formulate an answer with the available data, or if the input is not a question, simply answer the user in the way you find appropriate.
    """,
    agent=writer_agent,
    expected_output="The answer to the employee, in a direct but sympathetic way.")

##8 - Criando a crew
A crew (ou equipe) junta os agentes e as tarefas com o objetivo de gerar um resultado final.

In [ ]:
crew = Crew(
      agents=[
        retriever_agent, writer_agent
      ],
      tasks=[summarize_task, get_data_task, answer_task],
      verbose=True
    )

2024-09-03 17:49:53,266 - 52296 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


##9 - Testando o processo

In [ ]:
result = crew.kickoff(inputs={"question":"How many tickets on the GoPro Hero have we got?"})

 [2024-09-03 17:40:11][DEBUG]: == Working Agent: Internal Support Specialist
 [2024-09-03 17:40:11][INFO]: == Starting Task: Interpret the user's input: "How many tickets on the GoPro Hero have we got?". Decide whether to pass the question to the data retriever to get references to answer the question or not. If the user's input is a valid question, output the question. If it isn't, and therefore the data retriever shouldn't be contacted, simply output an empty string.
    


> Entering new CrewAgentExecutor chain...
Thought: The user's input is a valid question related to internal support, specifically about ticket volume for a product. This question requires data retrieval, so I should ask the Senior Data Retriever for this information.

Action: Ask question to coworker

Action Input: {"question": "How many tickets on the GoPro Hero have we got?", "context": "An employee has asked about the number of tickets related to the GoPro Hero product. We need to retrieve this information from

In [ ]:
print(result)

We currently have 228 tickets related to the GoPro Hero in our Support Tickets Database. Is there anything specific about these tickets you'd like to know?


In [ ]:
result = crew.kickoff(inputs={"question":"What to do if the user reports missing credits?"})

 [2024-09-03 17:50:00][DEBUG]: == Working Agent: Internal Support Specialist
 [2024-09-03 17:50:00][INFO]: == Starting Task: Interpret the user's input: "What to do if the user reports missing credits?". Decide whether to pass the question to the data retriever to get references to answer the question or not. If the user's input is a valid question, output the question. If it isn't, and therefore the data retriever shouldn't be contacted, simply output an empty string.
    


> Entering new CrewAgentExecutor chain...
To handle this task effectively, I need to determine if the user's input is a valid question that requires data retrieval or if it can be answered directly based on internal support knowledge.

The input "What to do if the user reports missing credits?" is a valid question that likely requires specific procedural information or guidelines. This type of question would benefit from retrieving accurate data to ensure a proper response to the employee.

Given that this is a va

In [ ]:
print(result)

If a user reports missing credits, here's what you should do:

1. Start by reviewing the user's usage logs and compare them with the invoiced amounts. This will help identify any discrepancies.

2. Check if there are any applied discounts or promotions that might affect the user's credit balance.

3. Verify that the correct subscription plan is applied to the user's account.

4. Confirm if there have been any recent plan changes or upgrades that could impact the credits.

Once you've completed these steps, explain each line item on the invoice in detail to the user. If you find any errors, apply the necessary corrections or credits to the account. Additionally, provide the user with a detailed usage report for the billing period to help them understand their account activity.

If the issue turns out to be complex or if there's a billing dispute that you can't resolve, escalate it to the Finance team for further assistance.

Remember to be patient and understanding with the user through